In [ ]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import csv
import pdb
import math
import time 
import pickle
import random
import math
import warnings
import datetime 
import collections
import numpy as np 
import pandas as pd  
import xgboost as xgb
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pylab as plt 
from matplotlib import rcParams   
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import time
import tensorflow
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from stellargraph.core import StellarGraph
from stellargraph.core import StellarDiGraph
from stellargraph.mapper import AdjacencyPowerGenerator
from stellargraph.layer import WatchYourStep
from stellargraph.losses import graph_log_likelihood
from stellargraph.utils import plot_history
from tensorflow.keras import optimizers, Model, layers, regularizers
from stellargraph.data import EdgeSplitter, UniformRandomWalk, UnsupervisedSampler
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.mapper.sampled_node_generators import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE, link_classification
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_path = "gdrive/My Drive/Major Project/"

In [ ]:
df_train = pd.read_csv(data_path+"pos_train_91.csv", index_col = 0)
print(df_train.shape)
df_train.head()

(452474, 3)


,src,dest,link
496205,54500,16485,1
141147,3318,30547,1
483066,33514,4007,1
478351,28118,2641,1
505144,69985,61687,1


In [ ]:
combined_tr = pd.read_csv(data_path+"/train_91.csv", index_col = 0)
print(combined_tr.shape)
combined_tr.head()

(902474, 3)


,src,dest,link
73132,35080,10701,0
404654,54809,75095,0
310475,67628,60404,0
301271,57253,462,0
332230,15167,47459,1


In [ ]:
combined_te = pd.read_csv(data_path+"/test_91.csv", index_col = 0)
print(combined_te.shape)
combined_te.head()

(106363, 3)


,src,dest,link
487942,32198,62377,0
103900,6056,11650,1
78661,2040,22467,1
71074,143,0,1
382438,1609,5035,1


In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())

In [ ]:
degrees = nx.degree(train_graph)
degrees = dict(degrees)

features = {}
for key, val in degrees.items():
  features[key] = [val]

pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

In [ ]:
nx.set_node_attributes(train_graph, features, "features")

In [ ]:
G = StellarGraph.from_networkx(train_graph, node_features="features")
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 75879, Edges: 452474

 Node types:
  default: [75879]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [452474]
        Weights: all 1 (default)
        Features: none


In [ ]:
nodes = list(G.nodes())
num_walks = 1
length = 5

unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length= length, number_of_walks = num_walks
)

In [ ]:
batch_size=128
epochs=20
num_samples= [10,5]

In [ ]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

In [ ]:
layer_sizes = [64,64]
graphsage = GraphSAGE(
    layer_sizes = layer_sizes, generator = generator, bias = True, normalize='l2'
)

x_inp, x_out = graphsage.in_out_tensors()

prediction = link_classification(
    output_dim = 1, output_act = "sigmoid", edge_embedding_method = "ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [ ]:
model = keras.Model(inputs = x_inp, outputs = prediction)
model.compile(
    optimizer = keras.optimizers.Adam(1e-2),
    loss = keras.losses.binary_crossentropy,
    metrics = [keras.metrics.binary_accuracy]
)
es = EarlyStopping(
    monitor = 'loss',
    patience = 2,
    verbose = 1,
    restore_best_weights = True
)

In [ ]:
history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=True,
    shuffle=True,
    callbacks = [es]
)
filep = open(data_path+"graphsage_model.h5", "wb")
pickle.dump(model, filep)
filep.close()

Epoch 1/20
4743/4743 [==============================] - 491s 103ms/step - loss: 0.6628 - binary_accuracy: 0.6080
Epoch 2/20
4743/4743 [==============================] - 484s 102ms/step - loss: 0.6533 - binary_accuracy: 0.6187
Epoch 3/20
4743/4743 [==============================] - 486s 102ms/step - loss: 0.6531 - binary_accuracy: 0.6200
Epoch 4/20
4743/4743 [==============================] - 483s 102ms/step - loss: 0.6527 - binary_accuracy: 0.6201
Epoch 5/20
4743/4743 [==============================] - 484s 102ms/step - loss: 0.6520 - binary_accuracy: 0.6207
Epoch 6/20
4743/4743 [==============================] - 485s 102ms/step - loss: 0.6516 - binary_accuracy: 0.6211
Epoch 7/20
4743/4743 [==============================] - 485s 102ms/step - loss: 0.6513 - binary_accuracy: 0.6209
Epoch 8/20
4743/4743 [==============================] - 485s 102ms/step - loss: 0.6519 - binary_accuracy: 0.6197
Epoch 9/20
4743/4743 [==============================] - 482s 102ms/step - loss: 0.6512 - binary_

In [ ]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [ ]:
node_ids = list(sorted(G.nodes()))
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

In [ ]:
node_emb = embedding_model.predict(node_gen, verbose=1)

593/593 [==============================] - 63s 106ms/step


In [ ]:
emb_dict = {}
for i, node in enumerate(node_ids):
  emb_dict[node] = np.array(node_emb[i].reshape(1,64))

In [ ]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined_tr.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)
print(train_emb.shape)
train_y = combined_tr['link']
print(len(train_y))

0it [00:00, ?it/s]

(902474, 64)
902474


In [ ]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_te.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])

print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_te['link'] 
print(len(test_y))

0it [00:00, ?it/s]

<class 'list'>
(106363, 64)
106363


In [ ]:
pickle.dump((train_emb, train_y), open(data_path + 'epinions_gsage_train91.pkl', 'wb'))
pickle.dump((test_emb, test_y), open(data_path + 'epinions_gsage_test91.pkl', 'wb'))

In [ ]:
train_emb, train_y = pickle.load(open(data_path + 'epinions_gsage_train91.pkl', 'rb'))
test_emb, test_y = pickle.load(open(data_path + 'epinions_gsage_test91.pkl', 'rb'))

In [ ]:
# train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [ ]:
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for XGB fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for XGB fit =  1196.5139129161835
Time taken for train data precition =  5.39226508140564
Time taken for test data precition =  0.6525747776031494
Train Score 0.8992222723818122 test Score 0.8915805569714101


In [ ]:
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for RandomForest fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for RandomForest fit =  818.3765449523926
Time taken for train data precition =  9.804877996444702
Time taken for test data precition =  1.275120735168457
Train Score 0.8902871722941486 test Score 0.8821599657029954


In [ ]:
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for LGBM fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for LGBM fit =  42.691112995147705
Time taken for train data precition =  5.1190996170043945
Time taken for test data precition =  0.6015708446502686
Train Score 0.8901460611894391 test Score 0.8873633264280472
